In [2]:
import re
import string
import numpy
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Masking
from keras.layers import Dense
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [3]:
with open('normalTrafficTraining.txt') as f:
    arr = f.readlines()
    content = ''.join(arr)
    items = re.findall("^GET.*|POST.*$", content, re.MULTILINE)
    data = ""
    for item in items:
        temp_data = item.replace(" HTTP/1.1","").lower()
        data = data + temp_data + "\n"

In [4]:
chars = list(set(data + string.punctuation + string.ascii_lowercase + ' ' + string.digits))
VOCAB_SIZE = len(chars)
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}
SEQ_LENGTH = 10
#print(chars)
DATA_LENGTH = int(len(data) / SEQ_LENGTH) 
#print(DATA_LENGTH, len(data))

In [5]:
X = numpy.zeros((DATA_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
y = numpy.zeros((DATA_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, DATA_LENGTH):
    X_sequence = data[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = numpy.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = data[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = numpy.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

In [7]:
model = Sequential()
model.add(LSTM(50, input_shape=(None, VOCAB_SIZE), return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
model.fit(X, y, batch_size=1, verbose=1, nb_epoch=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/1
270749/270749 [==============================] - 1937s 7ms/step - loss: 0.6886


In [9]:
model.save('LSTMmodel.h5')

In [85]:
data=[]
with open('normalTrafficTest.txt') as f:
    arr = f.readlines()
    content = ''.join(arr)
    items = re.findall("^GET.*|POST.*$", content, re.MULTILINE)
    for item in items:
        temp_data = item.replace(" HTTP/1.1","").lower()
        data.append(temp_data)

In [87]:

for seq in data[:50]:
    print (seq)
    probability=[]
    X = numpy.zeros((1,len(seq),VOCAB_SIZE))
    for i in range(1,len(seq),1):
        ix = char_to_ix[seq[i-1]]
        X[0,i,:][ix]=1
        probability.append(model.predict(X[:,:i,:])[0][0][char_to_ix[seq[i]]])
        #X = numpy.zeros((1,len(seq),VOCAB_SIZE))
    #print(probability)
    for i in probability:
        if (i<0.0001):
            print (i)
    

get http://localhost:8080/tienda1/index.jsp
get http://localhost:8080/tienda1/publico/anadir.jsp?id=1&nombre=jam%f3n+ib%e9rico&precio=39&cantidad=41&b1=a%f1adir+al+carrito
post http://localhost:8080/tienda1/publico/anadir.jsp
get http://localhost:8080/tienda1/publico/autenticar.jsp?modo=entrar&login=caria&pwd=egipciaca&remember=off&b1=entrar
post http://localhost:8080/tienda1/publico/autenticar.jsp
get http://localhost:8080/tienda1/publico/caracteristicas.jsp?id=2
post http://localhost:8080/tienda1/publico/caracteristicas.jsp
get http://localhost:8080/tienda1/publico/carrito.jsp
get http://localhost:8080/tienda1/publico/entrar.jsp?errormsg=credenciales+incorrectas
post http://localhost:8080/tienda1/publico/entrar.jsp
get http://localhost:8080/tienda1/publico/miembros.jsp
get http://localhost:8080/tienda1/publico/pagar.jsp?modo=insertar&precio=3230&b1=pasar+por+caja
post http://localhost:8080/tienda1/publico/pagar.jsp
get http://localhost:8080/tienda1/publico/productos.jsp
get http://lo

In [ ]:
#ix = [numpy.random.randint(VOCAB_SIZE)]
#length = 100
#y_char = [ix_to_char[ix[-1]]]
#X = numpy.zeros((1,length,VOCAB_SIZE))
#for i in range(5):
#    X[0,i,:][ix[-1]]=1
#    print(model.predict(X[:,:i+1,:]))